<img src='https://radiant-assets.s3-us-west-2.amazonaws.com/PrimaryRadiantMLHubLogo.png' alt='Radiant MLHub Logo' width='300'/>

# South Africa Field Boundary Detection Tutorial

## Model Training

This notebook focuses on training the South Africa field boundary data using UNet-Agri. With UNet-Agri, we will build a UNet model and take the approach further by adding a pre-trained Visual Geometry Group-19 (VGG-19) encoder with it to improve its performance.

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0' #gpu

In [1]:
#setting up the data path
from pathlib import Path
downloads_path = str(Path().resolve())
data_path =str(f"{downloads_path}/data")

### Importing necessary libraries

In [2]:
import os
import sys
import random

import cv2
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from PIL import Image

from tqdm import tqdm #for checking time progress of notebook cell
from itertools import chain
#for reading and shaping images
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.morphology import label
#splitting the data
from sklearn.model_selection import train_test_split

import segmentation_models as sm
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.applications import * #load vgg for u-net backbone


Segmentation Models: using `keras` framework.


### Data Preparation for Training

We will then load the data as `X` (for images to be trained on) and `Y` (for masks).

As we still had loads of augmented images in the training data from the previous step, we decided to use the saved training data for training and validation.

In [ ]:
augmented_path = data_path+"/augmented/" #augmented data from previous notebook
train_frame_path = f"{augmented_path}/train_frames" #decided to use the train data for train and val as it is sufficient enough
train_mask_path = f"{augmented_path}/train_masks"
train_ids = os.listdir(train_frame_path)
print(len(train_ids))

In [ ]:
IMG_WIDTH = 256 
IMG_HEIGHT = 256 
IMG_CHANNELS = 3
warnings.filterwarnings('ignore', category=UserWarning, module='skimage') #neglect minor warnings
seed = 8

Having 23000+ images for model training is an overkill, so we'll just use 7000 images as seen below:

In [ ]:
X = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8) #empty numpy array for storing images
print('Getting and resizing train images... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = train_frame_path #for training & validation
    imge = imread(path+"/" + id_ )[:,:,:IMG_CHANNELS]
    #imge = resize(imge, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True) #can use this to resize into a different dimension
   # imge = resize(imge, (IMG_HEIGHT, IMG_WIDTH))
    X[n] = imge
    if n == 7000: #7000 images. can be adjusted to desired amount depending on computational limits
        break

In [ ]:
Y = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.float32)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.float32)
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    #mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    path = train_mask_path
    mask_ = imread(path+"/" + id_)/255.
    mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True), axis=-1)
    mask = np.maximum(mask, mask_)
#         mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
#                                       preserve_range=True), axis=-2)
        
    Y[n] = mask_
    if n == 7000:
        break

In [ ]:
X=X[0:7000]
Y=Y[0:7000]

Calling `model.fit` automatically splits this data into training and validation data for model training. So all we need to do is split `X` and `Y` into `x, y` and `x_test, y_test` for training and test data respectively using a 20% split.

In [ ]:
from sklearn.model_selection import train_test_split
x, x_test, y, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
#visualise a sample from the train dataset
imshow(x[3]*2)
plt.show()
imshow(y[3])
plt.show()

### Building the U-Net Agri (U-Net with a pre-trained VGG19) Model

In [ ]:
######## Setting up U-Net with pre-trained VGG19 #########

#helpful resources to understand the model: https://arxiv.org/pdf/1505.04597.pdf
#https://arxiv.org/pdf/2006.04868.pdf

def squeeze_excite_block(inputs, ratio=8): #for feature re-calibration
    #improves performance by not losing distinguishing features
    init = inputs
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    #he-normal takes into account the non-linearity of relu activation function and sigmoid
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    x = Multiply()([init, se])
    return x

def conv_block(inputs, filters):
    #performs feature extraction
    x = inputs

    x = Conv2D(filters, (3, 3), padding="same")(x) #produces tensor of outputs
    x = BatchNormalization()(x)#takes conv2d output and normalises them. faster
    x = Activation('relu')(x)

    x = Conv2D(filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = squeeze_excite_block(x)

    return x

def encoder1(inputs): #vgg encoder for the unet
    skip_connections = []

    model = VGG19(include_top=False, weights='imagenet', input_tensor=inputs) #APPLYING VGG19 ENCODER
    names = ["block1_conv2", "block2_conv2", "block3_conv4", "block4_conv4"]
    for name in names:
        skip_connections.append(model.get_layer(name).output)

    output = model.get_layer("block5_conv4").output
    return output, skip_connections

def decoder1(inputs, skip_connections): #u-net decoder
    num_filters = [256, 128, 64, 32]
    skip_connections.reverse()
    x = inputs

    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation='bilinear')(x)
        x = Concatenate()([x, skip_connections[i]])
        x = conv_block(x, f)

    return x

def encoder2(inputs):
    num_filters = [32, 64, 128, 256]
    skip_connections = []
    x = inputs

    for i, f in enumerate(num_filters):
        x = conv_block(x, f)
        skip_connections.append(x)
        x = MaxPool2D((2, 2))(x)

    return x, skip_connections

def decoder2(inputs, skip_1, skip_2):
    num_filters = [256, 128, 64, 32]
    skip_2.reverse()
    x = inputs

    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation='bilinear')(x)
        x = Concatenate()([x, skip_1[i], skip_2[i]])
        x = conv_block(x, f)

    return x

def output_block(inputs):
    x = Conv2D(1, (1, 1), padding="same")(inputs)
    x = Activation('sigmoid')(x)
    return x

def Upsample(tensor, size):
    """Bilinear upsampling"""
    def _upsample(x, size):
        return tf.image.resize(images=x, size=size)
    return Lambda(lambda x: _upsample(x, size), output_shape=size)(tensor)

def ASPP(x, filter):
    shape = x.shape

    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(x)
    y1 = Conv2D(filter, 1, padding="same")(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation='bilinear')(y1)

    y2 = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(x)
    y2 = BatchNormalization()(y2)
    y2 = Activation("relu")(y2)

    y3 = Conv2D(filter, 3, dilation_rate=6, padding="same", use_bias=False)(x)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)

    y4 = Conv2D(filter, 3, dilation_rate=12, padding="same", use_bias=False)(x)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)

    y5 = Conv2D(filter, 3, dilation_rate=18, padding="same", use_bias=False)(x)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)

    y = Concatenate()([y1, y2, y3, y4, y5])

    y = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    return y

def build_model(shape):
    inputs = Input(shape)
    x, skip_1 = encoder1(inputs)
    x = ASPP(x, 64)
    x = decoder1(x, skip_1)
    outputs1 = output_block(x)

    x = inputs * outputs1

    x, skip_2 = encoder2(x)
    x = ASPP(x, 64)
    x = decoder2(x, skip_1, skip_2)
    outputs2 = output_block(x)
    outputs = Concatenate()([outputs1, outputs2])

    model = Model(inputs, outputs)
    return model

In [ ]:
model = build_model((256, 256, 3)) #IMAGE SIZE, build model
model.summary()

In [ ]:
#using iouscore, f1score, recall and precision metrics for our model
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5),
          sm.metrics.Recall(threshold=0.5), sm.metrics.Precision(threshold=0.5)]  #applied the threshold
#loss=sm.losses.dice_loss + sm.losses.bce_jaccard_loss
model.compile(optimizer='adam', loss=sm.losses.binary_crossentropy+sm.losses.dice_loss, metrics=metrics)#dice loss and binary crossentropy

In [ ]:
tf.config.experimental_run_functions_eagerly(True)

### Model Training

In [ ]:
batch_size=4
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#earlystopper = EarlyStopping(patience=3, verbose=1) #stopper after number of epochs to end if no improvements, did not apply initially
checkpointer = ModelCheckpoint(f"{data_path}/unet_agri_256x256.h5", verbose=1, save_best_only=True) #save best result as model
history = model.fit(x=x_train, y=y_train,
              validation_data=(x_val, y_val),
              steps_per_epoch = len(x_)//batch_size,
              validation_steps = len(x_val)//batch_size,
              batch_size=batch_size, epochs=40, callbacks=[callbacks=[checkpointer]])

### Plot training results on a graph

In [ ]:
############# Plot training results ##############
#training results
loss = results.history['loss'] 
iou_score = results.history['iou_score']
f1_score = results.history['f1-score']
precision = results.history['precision']
recall = results.history['recall']

#validation results
val_loss = results.history['val_loss']
val_iou_score = results.history['val_iou_score']
val_f1_score = results.history['val_f1-score']
val_precision = results.history['val_precision']
val_recall = results.history['val_recall']

In [ ]:
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

plt.plot(epochs, iou_score, 'bo', label='Training iou score')
plt.plot(epochs, val_iou_score, 'b', label='Validation iou score')
plt.title('Training and validation iou score')
plt.legend()
plt.figure()

plt.plot(epochs, f1_score, 'bo', label='Training f1 score')
plt.plot(epochs, val_f1_score, 'b', label='Validation f1 score')
plt.title('Training and validation f1 score')
plt.legend()
plt.figure()

plt.plot(epochs, precision, 'bo', label='Training precision score')
plt.plot(epochs, val_precision, 'b', label='Validation precision score')
plt.title('Training and validation precision score')
plt.legend()
plt.figure()

plt.plot(epochs, recall, 'bo', label='Training recall score')
plt.plot(epochs, val_recall, 'b', label='Validation recall score')
plt.title('Training and validation recall score')
plt.legend()
plt.figure()

plt.show()